# Project Details

### AIM : 
<b> To implement a language model which will perform better in playing the Hangman game as compared to the traditional models which depend on unigram,bigram and n-gram models</b>

### Personal Details

<ol>
    <li>Name: <b>G V Sandeep</b></li>
    <li>College: <b>BITS - Pilani Hyderabad Campus</b></li>
    <li>Github: <a href="https://github.com/greetsandeep/">greetsandeep</a></li>
</ol>

This code is open sourced and can be found at : <a href""></a>